#ClassificationLowestBRToken

# Install Libraries and imports

In [1]:
!pip install fastai -Uqq

     |████████████████████████████████| 189 kB 5.3 MB/s 
     |████████████████████████████████| 55 kB 1.9 MB/s 


In [2]:
from fastai.tabular.all import *
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import r2_score
from torch.utils.data import Dataset

# Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = Path('/content/drive/MyDrive/colab_notebooks/algovera/defi')

# Preparing the data

In [5]:
df = pd.read_csv(path/'compound.csv')
df["Date"] = pd.to_datetime(df["Timestamp"], unit='s', origin='unix')

tokens = df["Token"].unique()
tokens

array(['DAI', 'USDC', 'USDT', 'ETH'], dtype=object)

In [6]:
df = df.drop_duplicates(['Timestamp', 'Token'])
counts = pd.DataFrame(df['Timestamp'].value_counts()).reset_index()
counts.columns = ['Timestamp', 'Counts']
df = df.merge(counts, on='Timestamp')

In [7]:
df = df[df['Counts'] == 4].reset_index(drop=True).drop('Counts', axis=1)

In [8]:
df1 = pd.DataFrame()
for tok in tokens:
    df_tok = df[df['Token']==tok]
    df_tok = df_tok.drop(['Token', 'Date'], axis=1)

    col_names = []
    for col in df_tok.columns:
        if col == 'Timestamp':
            col_names.append(f'{col}')
        else:
            col_names.append(f'{tok}_{col}')
        
    df_tok.columns = col_names
    #df_tok = df_tok.set_index('Timestamp', drop=True)
    
    if df1.empty:
        df1 = df_tok
    else:
        df1 = pd.merge(df1, df_tok, on='Timestamp')

In [9]:
df1.sort_values('Timestamp', inplace=True)
df1["Date"] = pd.to_datetime(df1["Timestamp"], unit='s', origin='unix')

In [10]:
df1.head()

,Timestamp,DAI_Borrowing Rate,DAI_Deposit Rate,DAI_Borrow Volume,DAI_Supply Volume,USDC_Borrowing Rate,USDC_Deposit Rate,USDC_Borrow Volume,USDC_Supply Volume,USDT_Borrowing Rate,USDT_Deposit Rate,USDT_Borrow Volume,USDT_Supply Volume,ETH_Borrowing Rate,ETH_Deposit Rate,ETH_Borrow Volume,ETH_Supply Volume,Date
0,1609471800,0.073195,0.050982,1.069964e+09,6.196481e+10,0.087046,0.066993,7.285430e+08,4.063042e+10,0.099588,0.077548,6.430536e+07,3.696225e+09,0.022952,0.000489,30553.654354,5.663257e+07,2021-01-01 03:30:00
1,1609473600,0.073101,0.050912,1.069961e+09,6.197050e+10,0.087053,0.066998,7.285469e+08,4.063019e+10,0.094890,0.073569,6.407877e+07,3.700299e+09,0.022952,0.000489,30553.703955,5.663353e+07,2021-01-01 04:00:00
2,1609475400,0.073061,0.050882,1.069972e+09,6.197354e+10,0.087058,0.067003,7.285528e+08,4.063019e+10,0.085767,0.065933,6.399418e+07,3.729213e+09,0.022951,0.000489,30553.830472,5.664534e+07,2021-01-01 04:30:00
3,1609477200,0.073436,0.051161,1.070496e+09,6.197966e+10,0.086921,0.066889,7.285713e+08,4.063660e+10,0.072946,0.055500,6.317162e+07,3.730028e+09,0.022952,0.000489,30553.786270,5.663860e+07,2021-01-01 05:00:00
4,1609479000,0.067829,0.047015,1.070566e+09,6.234580e+10,0.086312,0.066383,7.285755e+08,4.066162e+10,0.057764,0.043239,6.256095e+07,3.752927e+09,0.022952,0.000489,30553.841412,5.663440e+07,2021-01-01 05:30:00


# Utility Functions

In [15]:
def get_tabpandas_multi(
    df:pd.DataFrame, # Dataframe of the raw data 
    target_window:int, # Number of timepoints in the future to predict 
    n_timepoint:int, # Number of previous timepoints to be used as features   
    inference:bool=False, # Flag True for inference
):

    df = df.reset_index(drop=True)
    feature_cols = ['DAI_Borrowing Rate', 'DAI_Deposit Rate', 'DAI_Borrow Volume', 'DAI_Supply Volume', 
                    'USDC_Borrowing Rate', 'USDC_Deposit Rate', 'USDC_Borrow Volume', 'USDC_Supply Volume', 
                    'USDT_Borrowing Rate', 'USDT_Deposit Rate', 'USDT_Borrow Volume', 'USDT_Supply Volume',
                    'ETH_Borrowing Rate', 'ETH_Deposit Rate', 'ETH_Borrow Volume', 'ETH_Supply Volume']

    target_columns = ['DAI_Borrowing Rate', 'USDC_Borrowing Rate', 'USDT_Borrowing Rate']
    target = 'Target'

    cols_names = []
    for j in range(n_timepoint):
        for col in feature_cols:
            cols_names.append(f'{col}_t-{n_timepoint -j-1}')
    cols_names += [target]

    pairs = []
    for i, row in tqdm(df.iterrows()):
        if i < (len(df)-target_window-n_timepoint-1):
            features = df.loc[i:i+n_timepoint-1, feature_cols].values
            features = [item for sublist in features for item in sublist]

            val =  df.loc[i+n_timepoint: i+n_timepoint-1+target_window, target_columns].mean().idxmin()

            features += [val]
            pairs.append(features)

    df = pd.DataFrame(pairs, columns=cols_names).dropna().reset_index(drop=True)
    
    if not inference:
        #n_min = df[target].value_counts()[2]
        splits = RandomSplitter(seed=101)(range_of(df))

        cont_names = list(df.columns[:-1])
        y_names = target
        procs = [Categorify, FillMissing, Normalize]
        y_block = CategoryBlock()

        to = TabularPandas(df, procs=procs, cont_names=cont_names, y_names=y_names, y_block=y_block, splits=splits)
        dls = to.dataloaders(bs=128)

        return df, to, dls

    if inference:
        return df


@patch
def plot_confusion_matrix(
    self:ClassificationInterpretation, 
    normalize=False, 
    title='Confusion matrix', 
    cmap="Blues", 
    norm_dec=2,
    plot_txt=True, 
    **kwargs
):
    "Plot the confusion matrix, with `title` and using `cmap`."
    # This function is mainly copied from the sklearn docs
    cm = self.confusion_matrix()
    if normalize: cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig = plt.figure(**kwargs)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(self.vocab))
    plt.xticks(tick_marks, self.vocab, rotation=90)
    plt.yticks(tick_marks, self.vocab, rotation=0)

    if plot_txt:
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            coeff = f'{cm[i, j]:.{norm_dec}f}' if normalize else f'{cm[i, j]}'
            plt.text(j, i, coeff, horizontalalignment="center", verticalalignment="center", color="white" if cm[i, j] > thresh else "black")

    ax = fig.gca()
    ax.set_ylim(len(self.vocab)-.5,-.5)

    plt.tight_layout()
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.grid(False)

    return fig


def get_preds_save_classification(
    learn, # fastai learner
    model_type, # LSTM model or fastai TabularModel
    n_timepoint:int, # Number of previous timepoints to be used as features   
    target_window:int, # Number of timepoints in the future to predict 
):
    preds, targs = learn.get_preds(dl=learn.dls.valid)
    targs = targs.squeeze()
    preds = torch.argmax(preds, 1)
    acc = sum(preds==targs)/len(preds) * 100
    interp = ClassificationInterpretation.from_learner(learn)
    p = interp.plot_confusion_matrix(title=f'{model_type} n_timepoint {n_timepoint} target_window {target_window} accuracy {acc}')
    p.savefig(f'{path}/results/{model_type} n_timepoint {n_timepoint} target_window {target_window}.png')


def get_tab_learner_train(
    df_p:pd.DataFrame, # Prepared df
    model_type:str, # LSTM or fastai TabularModel
    n_timepoint:int, # Number of previous timepoints to be used as features   
    target_window:int, # Number of timepoints in the future to predict 
):
    
    df, to, dls = get_tabpandas_multi(df_p, target_window, n_timepoint)
    sm = SaveModelCallback(fname=f'{path}/models/{model_type}_{n_timepoint}_{target_window}')

    learn = tabular_learner(dls, 
                        [200,100], 
                        metrics=accuracy,
                        n_out=3,
                        cbs=sm)
    
    learn.fit_one_cycle(100, 0.03)
    
    get_preds_save_classification(learn, model_type, target_window, n_timepoint)
    
    return learn

class LSTMDataset(Dataset):
    def __init__(self, 
        df, # `to` from fastai tabular 
        n_timepoint:int, # Number of previous timepoints to be used as features   
        numfeatpertimepoint:int, # Number of features per timepoint 
    ):
        self.df = df.items.reset_index(drop=True)
        self.n_sequence = n_timepoint
        self.numfeatures = numfeatpertimepoint
        self.vocab = ([0, 1, 2], )

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.loc[index,:]
        target = tensor(row.iloc[-1]).long()
        features = [tensor(list(row[i*self.numfeatures:(i*self.numfeatures+self.numfeatures)].values)) for i in range(self.n_sequence)]
        features = torch.stack(features)
        return (features, target)


class LSTMModel(nn.Module):
    def __init__(self, 
        input_size:int, # Number of features per timepoint
        hidden_size:int, # Hidden size to be used 
        num_layers:int=2, # Number of LSTM layers to use
        num_classes:int=3, # Output size
    ):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :]
        out = self.fc(out)

        return out


def get_lstm_learner_train(
    model_type:str, # LSTM or fastai TabularModel
    n_timepoint:int, # Number of previous timepoints to be used as features   
    target_window:int, # Number of timepoints in the future to predict 
):
    
    df, to, dls = get_tabpandas_multi(df1, target_window, n_timepoint)
    train_dset = LSTMDataset(to.train, n_timepoint, 16)
    valid_dset = LSTMDataset(to.valid, n_timepoint, 16)

    dls2 = DataLoaders.from_dsets(train_dset, valid_dset, bs=128)
    
    hidden_size = 128
    
    sm = SaveModelCallback(fname=f'{path}/models/{model_type}_{n_timepoint}_{target_window}')
    model = LSTMModel(16, hidden_size)
    learn = Learner(dls2, 
                model, 
                metrics=accuracy, 
                loss_func=CrossEntropyLossFlat(),
                cbs=sm)
    learn.fit_one_cycle(50, 0.03)    
    get_preds_save_classification(learn, model_type, n_timepoint, target_window)
    
    return df, dls, dls2, learn

## TabModel Classification

In [16]:
target_window = 3
n_timepoint = 1
model_type = 'TabClassificationLowestBRToken'

In [18]:
# learn = get_tab_learner_train(df1, model_type, n_timepoint, target_window)

## LSTM

In [ ]:
model_type = 'LSTMClassificationLowestBRToken'

In [ ]:
for ntp in [1, 5, 10, 20, 48, 48*3]:
    for tw in [1, 3, 5, 10, 20, 48, 48*3]:
        df, dls, dls2, learn = get_lstm_learner_train(model_type, ntp, tw)

13766it [00:26, 514.80it/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.743658,0.565641,0.751453,00:05
1,0.385514,0.225791,0.933140,00:05
2,0.216778,0.219926,0.917878,00:05
3,0.187445,0.290232,0.910247,00:05
4,0.169358,0.179907,0.940044,00:06
5,0.166931,0.188023,0.928052,00:05
6,0.190344,0.190472,0.920058,00:05
7,0.179686,0.202076,0.933503,00:05
8,0.185933,0.253614,0.912427,00:05
9,0.182065,0.185859,0.944404,00:05


Better model found at epoch 0 with valid_loss value: 0.5656414031982422.
Better model found at epoch 1 with valid_loss value: 0.22579139471054077.
Better model found at epoch 2 with valid_loss value: 0.21992574632167816.
Better model found at epoch 4 with valid_loss value: 0.17990702390670776.
Better model found at epoch 12 with valid_loss value: 0.16198818385601044.
Better model found at epoch 21 with valid_loss value: 0.16197766363620758.
Better model found at epoch 22 with valid_loss value: 0.1554548442363739.
Better model found at epoch 30 with valid_loss value: 0.15504619479179382.
Better model found at epoch 31 with valid_loss value: 0.15491066873073578.
Better model found at epoch 33 with valid_loss value: 0.15370149910449982.
Better model found at epoch 34 with valid_loss value: 0.1535332053899765.


13766it [00:26, 519.46it/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.765597,0.583277,0.730378,00:05
1,0.403390,0.234197,0.918241,00:05
2,0.244141,0.212686,0.924419,00:05
3,0.201226,0.180171,0.932049,00:05
4,0.194287,0.211365,0.923692,00:05
5,0.214794,0.223108,0.921875,00:05
6,0.232783,0.214980,0.916788,00:05
7,0.212231,0.196931,0.933140,00:05
8,0.220301,0.197554,0.931323,00:05
9,0.221961,0.226061,0.914608,00:05


...

epoch,train_loss,valid_loss,accuracy,time
0,0.589806,0.449474,0.815261,00:30
1,0.456325,0.416590,0.817452,00:31
2,0.371163,0.341485,0.856152,00:30
3,0.359740,0.298730,0.880613,00:30
4,0.348835,0.298843,0.878058,00:30
5,0.352972,0.434504,0.830960,00:30
6,0.361123,0.311819,0.865644,00:30
7,0.310581,0.225980,0.912012,00:30
8,0.347794,0.307976,0.872946,00:30
9,0.338664,0.375357,0.856882,00:30


Better model found at epoch 0 with valid_loss value: 0.44947385787963867.
Better model found at epoch 1 with valid_loss value: 0.4165899157524109.
Better model found at epoch 2 with valid_loss value: 0.34148478507995605.
Better model found at epoch 3 with valid_loss value: 0.2987295389175415.
Better model found at epoch 7 with valid_loss value: 0.22598040103912354.
Better model found at epoch 15 with valid_loss value: 0.21349482238292694.
Better model found at epoch 16 with valid_loss value: 0.18363149464130402.
Better model found at epoch 21 with valid_loss value: 0.1652565896511078.
Better model found at epoch 23 with valid_loss value: 0.14509917795658112.
Better model found at epoch 25 with valid_loss value: 0.13526402413845062.
Better model found at epoch 26 with valid_loss value: 0.1033531129360199.
Better model found at epoch 27 with valid_loss value: 0.09966287761926651.
Better model found at epoch 30 with valid_loss value: 0.09257739037275314.
Better model found at epoch 31 wit

13766it [00:29, 458.89it/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.610284,0.465935,0.810662,00:30
1,0.413044,0.365775,0.848529,00:30
2,0.364412,0.392315,0.831985,00:29
3,0.353975,0.348903,0.854412,00:30
4,0.310694,0.398154,0.830147,00:30
5,0.379216,0.423339,0.842647,00:30
6,0.356490,0.477872,0.800735,00:30
7,0.370282,0.397986,0.822794,00:30
8,0.364002,0.461243,0.800735,00:30
9,0.402752,0.436009,0.778676,00:30


Better model found at epoch 0 with valid_loss value: 0.4659348428249359.
Better model found at epoch 1 with valid_loss value: 0.36577504873275757.
Better model found at epoch 3 with valid_loss value: 0.3489031195640564.
Better model found at epoch 12 with valid_loss value: 0.3108261227607727.
Better model found at epoch 13 with valid_loss value: 0.2790660858154297.
Better model found at epoch 14 with valid_loss value: 0.2709570527076721.
Better model found at epoch 16 with valid_loss value: 0.18383672833442688.
Better model found at epoch 17 with valid_loss value: 0.15318289399147034.
